In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [1]:
import torch
import pretrained
import pytorch_memlab
from actuallysparse import converter
from tqdm import tqdm
from warnings import filterwarnings

filterwarnings("ignore")

device = "cuda" if torch.cuda.is_available() else "cpu"
model_loader_very_pruned = lambda: torch.load(".weights/full/very_pruned").eval()
model_loader_pruned = lambda: torch.load(".weights/full/pruned").eval()
model_loader_pretrained = lambda: torch.load(".weights/full/pretrained").eval()

def features():
    print()
    very_pruned = model_loader_pruned().features
    return very_pruned

def dense_classifier():
    print()
    very_pruned = model_loader_very_pruned().classifier
    return very_pruned

def very_pruned_classifier_coo():
    print()
    very_pruned_coo = converter.convert_model(model_loader_very_pruned(), torch.nn.Linear, "coo").classifier
    return very_pruned_coo

def pruned_classifier_coo():
    print()
    very_pruned_coo = converter.convert_model(model_loader_pruned(), torch.nn.Linear, "coo").classifier
    return very_pruned_coo



with pytorch_memlab.LineProfiler(
        dense_classifier,
        very_pruned_classifier_coo,
        features,
        pruned_classifier_coo
    ) as prof:
    features()
    dense_classifier()
    pruned_classifier_coo()
    very_pruned_classifier_coo()

prof.display()

active_bytes,reserved_bytes,line,code
all,all,,
peak,peak,,
35.22M,126.00M,20,def dense_classifier():
0.00B,126.00M,21,print()
107.41M,126.00M,22,very_pruned = model_loader_very_pruned().classifier
72.19M,126.00M,23,return very_pruned
active_bytes,reserved_bytes,line,code
all,all,,
peak,peak,,
36.51M,492.00M,25,def very_pruned_classifier_coo():


In [3]:
def eval_accuracy(model, dataloader):
    with torch.no_grad():
        model.to(device)
        correct = 0
        all_so_far = 0
        for inputs, labels in tqdm(dataloader):
            inputs, labels = inputs.to(device), labels.to(device)
            pred = torch.argmax(model(inputs), dim=1)

            all_so_far += labels.size().numel()
            correct += torch.sum(pred.eq(labels))
    return correct/all_so_far

_, dataloader_test = pretrained.load_cifar10_dataloaders()

Files already downloaded and verified
Files already downloaded and verified


In [13]:
eval_accuracy(model_loader_pretrained(), dataloader_test)

100%|██████████| 625/625 [00:09<00:00, 66.45it/s]


tensor(0.9074, device='cuda:0')

In [14]:
eval_accuracy(model_loader_pruned(), dataloader_test)

100%|██████████| 625/625 [00:09<00:00, 65.45it/s]


tensor(0.8365, device='cuda:0')

In [15]:
eval_accuracy(model_loader_very_pruned(), dataloader_test)

100%|██████████| 625/625 [00:09<00:00, 64.36it/s]


tensor(0.7648, device='cuda:0')

In [7]:
dense = model_loader_very_pruned().to(device).eval()

In [8]:
%%time
eval_accuracy(dense, dataloader_test)

100%|██████████| 625/625 [00:09<00:00, 66.61it/s]

CPU times: user 9.36 s, sys: 23.7 ms, total: 9.38 s
Wall time: 9.39 s


tensor(0.7648, device='cuda:0')

In [11]:
dense_cpu = dense.cpu()

In [12]:
%%time
eval_accuracy(dense_cpu, dataloader_test)

100%|██████████| 625/625 [00:09<00:00, 64.50it/s]

CPU times: user 9.63 s, sys: 50.8 ms, total: 9.68 s
Wall time: 9.72 s


tensor(0.7648, device='cuda:0')

In [16]:
coo = converter.convert_model(model_loader_pruned(), torch.nn.Linear, "coo").to(device)
coo.train_mode = False # actually sparse

In [17]:
%%time
eval_accuracy(coo, dataloader_test)

100%|██████████| 625/625 [00:14<00:00, 42.99it/s]

CPU times: user 14.5 s, sys: 28.6 ms, total: 14.5 s
Wall time: 14.5 s


tensor(0.8365, device='cuda:0')

In [18]:
coo_cpu = coo.cpu()

In [19]:
%%time
eval_accuracy(coo_cpu, dataloader_test)

100%|██████████| 625/625 [00:14<00:00, 42.96it/s]

CPU times: user 14.5 s, sys: 45 ms, total: 14.6 s
Wall time: 14.6 s


tensor(0.8365, device='cuda:0')

In [20]:
coo = converter.convert_model(model_loader_very_pruned(), torch.nn.Linear, "coo").to(device)
coo.train_mode = False # actually sparse

In [21]:
%%time
eval_accuracy(coo, dataloader_test)

100%|██████████| 625/625 [00:11<00:00, 55.18it/s]

CPU times: user 11.3 s, sys: 24.1 ms, total: 11.3 s
Wall time: 11.3 s


tensor(0.7648, device='cuda:0')

In [22]:
coo_cpu = coo.cpu()

In [23]:
%%time
eval_accuracy(coo_cpu, dataloader_test)

100%|██████████| 625/625 [00:11<00:00, 55.22it/s]

CPU times: user 11.3 s, sys: 28.2 ms, total: 11.3 s
Wall time: 11.3 s


tensor(0.7648, device='cuda:0')